In [3]:
#Reference : https://wikidocs.net/22893
import pandas as pd
import tensorflow as tf
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

tf.debugging.set_log_device_placement(True)

In [4]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [5]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

178009

In [6]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
31638,Tell it to the cops.,Dis-le aux policiers.
54020,Everyone seems to agree.,Tout le monde semble être d'accord.
21846,We have no chance.,Nous n'avons aucune chance.
35229,I can squeeze you in.,Je peux t'insérer.
55671,I saw tears in his eyes.,Je vis des larmes dans ses yeux.
53544,Can I talk to you a sec?,Puis-je te parler une seconde ?
27353,What happened here?,Qu'est-ce qu'il s'est passé ici ?
56573,Is this sterling silver?,Est-ce de l'argent fin?
44557,The snow is very deep.,La neige est très profonde.
26881,Tom needs our help.,Tom a besoin de notre coup de main.


In [7]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
20378,It's one of those.,\t C'est l'un d'eux. \n
41785,"I don't hate you, Tom.","\t Je ne te hais pas, Tom. \n"
9903,You're careful.,\t Tu es prudent. \n
48136,I can't just stay here.,\t Je ne peux pas juste rester ici. \n
41529,I admire your bravery.,\t J'admire votre bravoure. \n
28907,He can help you out.,\t Il peut vous donner un coup de main. \n
20333,It's dark in here.,\t Il fait noir là-dedans. \n
17150,We weren't ready.,\t Nous n'étions pas prêtes. \n
59121,What we want is freedom.,"\t Ce que nous voulons, c'est la liberté. \n"
47675,He may have told a lie.,\t Peut-être a-t-il menti. \n


In [8]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)